In [1]:
%matplotlib inline

import sys
import os
sys.path.append("../../..")

import numpy as np
import re
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import exposure, img_as_float, morphology, color
from skimage.measure import label
from sklearn.model_selection import KFold, LeavePOut
from PIL import Image
import xml.etree.ElementTree as ET

from osgeo import gdal
import osr
import my_packages.My_Geoprocess as mgp

from scipy.interpolate import griddata

import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

In [2]:
BathyFolder = '../../Resultats/Bathymetries/S2A_MSIL1C_20170929T231851_N0205_R001_T58KDE_20170929T231932/'
RasterName = 'Stumpf_NoClass_2ItersRidge_GlintProcess_Done_PixInterp_nearest_CoverPixRatio3%_CalibZone_Lagoon.tif'


In [3]:
Bathy = gdal.Open(BathyFolder+RasterName, gdal.GA_ReadOnly)
BathyMap = Bathy.GetRasterBand(1).ReadAsArray()

In [4]:
ImagesFolder = '../../Images/Acolite/'+Bathy.GetMetadata()['ImageName']+'/'
Lagoon = 'Mask/Lagoon.tif'

ClassifIndex = gdal.Open(ImagesFolder + Lagoon, gdal.GA_ReadOnly)
ClassifIndex = ClassifIndex.GetRasterBand(1).ReadAsArray().astype(bool)

In [5]:
TofillIndex = np.logical_and(np.isnan(BathyMap), ClassifIndex)
r,c = np.where(~np.isnan(BathyMap))
Points = np.stack((r,c),axis=1)

r_i,c_i = np.where(TofillIndex)
Points_i = np.stack((r_i,c_i),axis=1)

Values = np.asarray([BathyMap[i,j] for i,j in zip(r,c)])
del r, c, r_i,c_i


In [6]:
InterPoints = griddata(points=Points, xi=Points_i, values=Values,method='linear')

In [7]:
InterBathy = np.full(BathyMap.shape, np.nan, BathyMap.dtype)
InterBathy[TofillIndex] = InterPoints
InterBathy[~TofillIndex] = BathyMap[~TofillIndex]
Raster = np.stack((BathyMap, Bathy.GetRasterBand(2).ReadAsArray(), InterBathy), axis=2)
del InterBathy, InterPoints, BathyMap

In [8]:
Metadata = Bathy.GetMetadata()
Metadata['Description'] = Metadata['Description'] + 'Band3: Bathymetry map with linearly interpolated value at missing location.'
Bathy.SetMetadata(Metadata)
GT, Proj = Bathy.GetGeoTransform(), Bathy.GetProjection()
Bathy = None

In [9]:

fname = BathyFolder + RasterName

mgp.write_raster2(fname, Raster, GT, Proj,  DriverName="GTiff", formatMem=gdal.GDT_Float32,Metadata=Metadata, Offset=None)
#response = gdal.Translate(destName=fname, srcDS=fTempName, srcWin=(0, 5500, 8500, 5480))
#if response is None:
#    print('fail')